# Epic Seven Gear Simulator

This is an ongoing project to create an app in python that will simulate the generation of items (better known as gear in the community), same as those found in the mobile game Epic Seven. This particular notebook is used for writing the test functions before moving them to their respective modules.

In [ ]:
import json
import random
import importlib

import sys
sys.path.append("./data")
sys.path.append("./tests")
sys.path.append("./src")

from src.validation_utils import *
from src.utilities import *
from src.stats import *
from tests.test_functions import *

# importlib.import_module('src.validation_utils')
# importlib.import_module('src.stats')
# importlib.import_module('tests.test_functions')

In [ ]:
# Import Data
import json

TYPES = json.loads(open('data/types.json', 'r').read())
GRADES = json.loads(open('data/grades.json', 'r').read())
SETS = json.loads(open('data/sets.json', 'r').read())
STATS = json.loads(open('data/stats.json', 'r').read())
TIERS = json.loads(open('data/tiers.json', 'r').read())

In [ ]:
def parse_stat(self, gear_grade=None, gear_level=85, mod=False,
               rolled=None, mod_type='greater'):
    """
    Parses a stat based on its type ('substat' or 'mainstat') and given gear grade and level.
    Also checks whether modification is being applied and parses the appropriate mod value based on rolled count.

    Args:
        gear_grade (str): grade of the gear - 'normal', 'good', 'rare', 'heroic', 'epic' (default: None)
        gear_level (int): level of gear, between 58 and 100 (default: 85)
        mod (bool): boolean specifying whether this is a modded stat or not (default: False)
        rolled (int): number of times a stat has been rolled when enhancing (default: None = 0)
        mod_type (str): type of modification - 'greater' or 'lower' (default: 'greater')

    Returns:
        dict: parsed Stat data
    """

    # Check if stat exists before trying to parse it
    if self.stat_id is None:
        raise ValueError("Cannot parse until a stat has been fetched.")

    # Validate inputs:
    gear_grade = validate_gear_grade(gear_grade)
    gear_level = validate_gear_level(gear_level)
    mod = validate_mod(mod)
    rolled = validate_rolled(rolled)
    mod_type = validate_mod_type(mod_type)

    # Get gear tier
    gear_tier = get_gear_tier(gear_level)

    # Get the corresponding stat from stats dict based on id
    stat = STATS[self.stat_id]

    # Get the text description
    self.text = stat['text']
    # Assign attributes
    self.gear_grade = gear_grade
    self.gear_level = gear_level
    self.gear_tier = gear_tier
    self.rolled = rolled

    # Access information from the stat object
    var = stat['vars'][self.stat_type]
    key = var['key']  # Get the key

    # Parse modified stat
    if mod:
        self.modded = True
        value = get_mod_value(
            self.stat_id,
            self.gear_level,
            self.rolled,
            mod_type)
    # Parse non-modified stat
    else:
        value = get_stat_value(
            self.stat_id,
            self.stat_type,
            self.gear_level,
            self.gear_grade)

    # Parsed attributes
    self.value = value
    self.value_key = STATS[self.stat_id]['vars'][self.stat_type]['key']

    # Reforge increase value
    self.reforge_increase = get_reforge_increase(
        self.stat_id, self.stat_type, self.rolled)

    return self

In [ ]:
import random

# Parse a stat (get a specific value for a stat from choices)


def parse_stat(stat, stat_type='mainstat', item_grade=None,
               mod=False, rolled=None, mod_type='greater'):    


    if mod:  # This section for modified stats

        # Variable to specify that this stat has been modded
        parsed_stat['modded'] = True

        if mod_type is not None and mod_type not in [
                'greater', 'lesser']:  # Check if we are provided appropriate values for mod_type
            print("Invalid mod_type provided. Defaulting to Greater Gem.")
            mod_type = 'greater'

        # Get the ranges of values for that mod_type
        values = stat['mod_vals'][mod_type]

        if item_level <= 88:  # If item level is <= 88, use the first index valus
            values_88 = values[0][rolled]
            value = random.choice(values_88)
        elif item_level == 90:  # For reforged items
            values_90 = values[1][rolled]
            value = random.choice(values_90)
        else:
            print("Invalid item level, cannot modify")
            return False

    else:  # This section for non modified stats
        # Variable to specify that this stat has not been modded yet
        parsed_stat['modded'] = False
        # Variable to store how many times a stat has been rolled (used for
        # reforging)
        parsed_stat['rolled'] = 0

        type_ = var['type']  # Get the type (fixed or random)
        values = var['values'][item_grade][item_tier - 5]  # Get the value(s)

        if type_ == 'rand':  # If the type is 'rand', calculate a random value based on the rates
            rates = var['rates'][item_grade][item_tier - 5]
            # Get a random value based on the rates
            value = random.choices(values, rates)[0]

        elif type_ == 'fixed':  # If the type is 'fixed', directly use the value for the tier
            value = values

    if (item_level == 85) or (item_level == 90):
        reforge_increase = get_reforge_increase(
            parsed_stat['id'], stat_type, rolled)
        # Variable to store the  reforged value
        parsed_stat['reforge_increase'] = reforge_increase

    # Append the parsed key-value pair to the 'values' list
    parsed_stat['values'].append([key, value])

    return parsed_stat

In [ ]:
import random

# Parse a stat (get a specific value for a stat from choices)


def parse_stat(stat, stat_type='mainstat', item_grade=None,
               item_level=85, mod=False, rolled=None, mod_type='greater'):

    # Check if stat_type and mod matches (cannot mod mainstat)
    if stat_type == 'mainstat' and mod:
        print("Mainstats cannot be modded.")
        return False

    # Check if correct parameters are passed for mod, it can only be True or
    # False (0 or 1)
    if mod is not None and mod not in [True, False]:
        print("Invalid mod status provided, defaulting to False.")
        mod = False

    # If no item grade is provided, assign a random item grade based on
    # crafting rates
    if item_grade is None:
        item_grade = get_random_grade()

    # Get Item tier from level
    item_tier = get_item_tier(item_level)

    parsed_stat = {}  # Initialize an empty dictionary to store parsed values
    parsed_stat['id'] = stat['id']  # Copy the 'id' from the input stat
    # Copy the 'text' description from the input stat
    parsed_stat['text'] = stat['text']
    # Copy the 'key_stat' from the input stat
    parsed_stat['key_stat'] = stat['key_stat']
    # Initialize an empty list to store parsed values
    parsed_stat['values'] = []

    var = stat['vars'][stat_type]  # Access information from the stat object
    key = var['key']  # Get the key

    # First, get the correct rolled value
    if rolled is not None:
        if 0 <= int(rolled) <= 5:
            rolled = int(rolled)
            # Variable to store how many times a stat has been rolled (used for
            # reforging)
            parsed_stat['rolled'] = rolled
        elif not isinstance(rolled, (int, float)) or rolled < 0 or rolled > 5:
            print("Roll count invalid. Defaulting to 0")
            parsed_stat['rolled'] = 0
    else:
        # print("Roll count not provided. Defaulting to 0")
        rolled = 0
        parsed_stat['rolled'] = rolled

    if mod:  # This section for modified stats

        # Variable to specify that this stat has been modded
        parsed_stat['modded'] = True

        if mod_type is not None and mod_type not in [
                'greater', 'lesser']:  # Check if we are provided appropriate values for mod_type
            print("Invalid mod_type provided. Defaulting to Greater Gem.")
            mod_type = 'greater'

        # Get the ranges of values for that mod_type
        values = stat['mod_vals'][mod_type]

        if item_level <= 88:  # If item level is <= 88, use the first index valus
            values_88 = values[0][rolled]
            value = random.choice(values_88)
        elif item_level == 90:  # For reforged items
            values_90 = values[1][rolled]
            value = random.choice(values_90)
        else:
            print("Invalid item level, cannot modify")
            return False

    else:  # This section for non modified stats
        # Variable to specify that this stat has not been modded yet
        parsed_stat['modded'] = False
        # Variable to store how many times a stat has been rolled (used for
        # reforging)
        parsed_stat['rolled'] = 0

        type_ = var['type']  # Get the type (fixed or random)
        values = var['values'][item_grade][item_tier - 5]  # Get the value(s)

        if type_ == 'rand':  # If the type is 'rand', calculate a random value based on the rates
            rates = var['rates'][item_grade][item_tier - 5]
            # Get a random value based on the rates
            value = random.choices(values, rates)[0]

        elif type_ == 'fixed':  # If the type is 'fixed', directly use the value for the tier
            value = values

    if (item_level == 85) or (item_level == 90):
        reforge_increase = get_reforge_increase(
            parsed_stat['id'], stat_type, rolled)
        # Variable to store the  reforged value
        parsed_stat['reforge_increase'] = reforge_increase

    # Append the parsed key-value pair to the 'values' list
    parsed_stat['values'].append([key, value])

    return parsed_stat

In [ ]:
def format_stat(parsed_stat, show_reforged=False, mainstat=False):

    # Get the original 'text' description from the parsed stat
    text = parsed_stat['text']
    if mainstat:
        reforged_value = parsed_stat['reforge_increase']
    else:
        reforged_value = parsed_stat['values'][0][1] + \
            parsed_stat['reforge_increase']

    # Iterate through the key-value pairs in the 'values' list of the parsed
    # stat
    for key, value in parsed_stat['values']:
        # Replace each occurrence of 'key' with 'value' in the text
        text = text.replace(key, str(value))

    if show_reforged:
        if parsed_stat['id'] in [1, 3, 5, 6, 7, 8, 9]:
            text = text.replace('<B>', '(' + str(reforged_value) + '%) ')
        else:
            text = text.replace('<B>', '(' + str(reforged_value) + ') ')
    else:
        text = text.replace('<B>', '')

    if parsed_stat['modded']:
        text = text + ' (modded)'

    # Return the formatted text representation of the parsed stat
    return text

In [ ]:
# Function to convert integers inside a list to a string
def convert_int_to_string(obj):

    if isinstance(obj, int):
        return [str(obj)]

    elif isinstance(obj, list):
        return [str(o) for o in obj]

    else:
        raise ValueError("Invalid input type. Expected int or list.")

# Function to check whether the provided substats are allowed in the
# item_type, if not, it rolls for a new item type


def check_type_subs(substat_ids, item_type):
    if substat_ids is not None:
        substat_ids = convert_int_to_string(substat_ids)
        valid_substats = TYPES[item_type]['substat']
        valid_substats = convert_int_to_string(valid_substats)
        while not all(
                substat_id in valid_substats for substat_id in substat_ids):
            item_type = random.choice(list(TYPES))
            # Update valid_substats for the new item_type
            valid_substats = TYPES[item_type]['substat']
            valid_substats = convert_int_to_string(valid_substats)
    return item_type

# Function to check whether the substat_ids are valid (must be a stat
# within our list and cannot be duplicates)


def check_valid_subs(substat_ids):

    # Convert to strings
    substat_ids = convert_int_to_string(substat_ids)

    # Get length of substats list (how many subs provided)
    len_ = len(substat_ids)

    # Check if more than 4 subs are provided:
    if len_ > 4:
        print("Please provide up to 4 sub stats only.")
        return False

    # Check if valid substat id's are provided
    for s in substat_ids:
        if str(s) not in list(STATS):
            print("Invalid substat provided.")
            return False

    # Check if duplicate substat id's are provided
    if len_ != len(set(substat_ids)):
        print("Duplicate substats provided.")
        return False

    else:
        return True

In [ ]:
import random
# This function creates n (iters) items and counts how many of each item
# type was generated along with rates


def count_item_types(item_type=None, mainstat_id=None, substat_ids=None,
                     item_grade=None, item_set=None, item_level=85, iters=1000):
    # Initialize empty lists:
    weapon = 0
    helm = 0
    armor = 0
    necklace = 0
    ring = 0
    boots = 0

    for i in range(0, iters):
        item = create_random_item(
            item_type=item_type,
            mainstat_id=mainstat_id,
            substat_ids=substat_ids,
            item_grade=item_grade,
            item_set=item_set,
            item_level=item_level)

        if item and item['type'] == 'Weapon':
            weapon += 1
        elif item and item['type'] == 'Helm':
            helm += 1
        elif item and item['type'] == 'Armor':
            armor += 1
        elif item and item['type'] == 'Necklace':
            necklace += 1
        elif item and item['type'] == 'Ring':
            ring += 1
        elif item and item['type'] == 'Boots':
            boots += 1

    print(
        f"Weapon: {weapon}, Helm: {helm}, Armor: {armor}, Necklace: {necklace}, Ring: {ring}, Boots: {boots}")

In [ ]:
def get_valid_item_type_from_subs(substat_ids, item_type):
    substat_ids = convert_int_to_string(substat_ids)  # Convert to strings
    # print(f"subs: {substat_ids}")
    valid_substats = TYPES[item_type]['substat']
    valid_substats = convert_int_to_string(valid_substats)
    # print(f"valids: {valid_substats}")
    # loop to checks if all the substat IDs in the substat_ids list are
    # present in the valid_substats list.
    while not all(substat_id in valid_substats for substat_id in substat_ids):
        item_type = random.choice(list(TYPES))
        valid_substats = TYPES[item_type]['substat']
        valid_substats = convert_int_to_string(valid_substats)

    return item_type

In [ ]:
import random


def create_random_item(item_type=None, mainstat_id=None,
                       substat_ids=None, item_grade=None, item_set=None, item_level=85):

    # If an Item Level out of our [58, 100] range is provided, default to 85
    if item_level < 58 or item_level > 100:
        print("Invalid item level provided, defaulting to 85.")
        item_level = 85
    # Get Item Tier
    item_tier = get_item_tier(item_level)

    # Check if proper item_type is provided:
    if item_type is not None and item_type not in list(TYPES):
        print("Invalid item type provided.")
        return False

    # Check if proper item_grade is provided:
    if item_grade is not None:
        if item_grade not in list(GRADES):
            print("Invalid item grade provided.")
            return False
        else:
            # Get the number of starting substats for this item
            # Number of substats the item will have
            starting_substats = GRADES[item_grade]['starting_substats']

    # Check if proper item_set is provided:
    if item_set is not None and item_set not in list(SETS):
        print("Invalid item set provided.")
        return False
    # Get a random Item Set if none provided
    elif item_set is None:
        item_set = random.choice(list(SETS))

    # Check that we have correct substats before proceeding:
    if substat_ids is not None:
        substat_ids = convert_int_to_string(substat_ids)
        # If not True, return False. If True, do nothing.
        if not check_valid_subs(substat_ids):
            return False

    # Check that we have correct mainstats before proceeding:
    if mainstat_id is not None:
        if str(mainstat_id) not in list(STATS):
            print("Invalid Mainstat provided.")
            return False

    # Initialize Empty lists
    mainstat = []  # Mainstats if None provided
    substats = []  # Substats if None provided

    # Get a random item_type if none is provided
    if mainstat_id is not None:
        if item_type is None:
            item_type = random.choice(list(TYPES))

            while int(mainstat_id) not in TYPES[item_type]['mainstat']:
                item_type = random.choice(list(TYPES))

            # If we have substats provided, we need to check if the item type
            # fits those given substats
            if substat_ids is not None:
                item_type = get_valid_item_type_from_subs(
                    substat_ids, item_type)
        else:
            if int(mainstat_id) not in TYPES[item_type]['mainstat']:
                print('This item type cannot have that mainstat!')
                return False
            if substat_ids is not None and any(s not in convert_int_to_string(
                    TYPES[item_type]['substat']) for s in substat_ids):
                print("This item type cannot have one or more of these substats!")
                return False
    else:
        if item_type is None:
            item_type = random.choice(list(TYPES))

            # Check whether this random item_type is allowed based on provided
            # substats
            if substat_ids is not None:
                item_type = get_valid_item_type_from_subs(
                    substat_ids, item_type)
        else:
            if substat_ids is not None and any(s not in convert_int_to_string(
                    TYPES[item_type]['substat']) for s in substat_ids):
                print("This item type cannot have one or more of these substats!")
                return False

    # Get appropriate item_grade if substats are provided
    # (e.g. if 4 substats are provided, the item must be an Epic item, so keep rolling grade until Epic is shown)
    if substat_ids is not None:

        # Check if mainstat and substats have same id's
        if mainstat_id is not None:
            if str(mainstat_id) in substat_ids:
                print("Mainstat and Substats cannot have the same stat.")
                return False

        no_of_subs = len(substat_ids)

        if item_grade is not None:
            if starting_substats < no_of_subs:
                print("Invalid item grade provided.")
                return False
        else:
            item_grade = get_random_grade()
            starting_substats = GRADES[item_grade]['starting_substats']
            while starting_substats < no_of_subs:
                item_grade = get_random_grade()
                starting_substats = GRADES[item_grade]['starting_substats']
    else:
        if item_grade is None:  # Get random item_grade if no substats are provided
            item_grade = get_random_grade()
            # Get the number of starting substats for this item
            # Number of substats the item will have
            starting_substats = GRADES[item_grade]['starting_substats']

    # Get Main Stat
    if mainstat_id is not None:  # If a mainstat is provided
        mainstat.append(
            parse_stat(
                get_stat_by_id(
                    str(mainstat_id)),
                'mainstat',
                item_grade,
                item_level))
    elif mainstat_id is None:  # If no mainstat is provided
        mainstat.append(
            parse_stat(
                get_random_stat(
                    'mainstat',
                    item_type),
                'mainstat',
                item_grade,
                item_level))

    # if Substats are provided
    if substat_ids is not None:
        for s in substat_ids:
            substats.append(
                parse_stat(
                    get_stat_by_id(s),
                    'substat',
                    item_grade,
                    item_level))
        # No of subs still needed to be added
        subs_to_add = starting_substats - no_of_subs
        for i in range(0, subs_to_add):
            stat = get_non_overlapping_stat(
                mainstat + substats, 'substat', item_type)
            substats.append(
                parse_stat(
                    stat,
                    'substat',
                    item_grade,
                    item_level))
    # If no substats are provided
    else:
        # Get first substat
        if starting_substats > 0:
            stat = get_non_overlapping_stat(mainstat, 'substat', item_type)
            # stat['rolled'] = 0 # Variable to store how many times the stat
            # has been rolled
            substats.append(
                parse_stat(
                    stat,
                    'substat',
                    item_grade,
                    item_level))

        # Add more subtats to fill the rest of the item:
        for i in range(1, starting_substats):
            stat = get_non_overlapping_stat(
                mainstat + substats, 'substat', item_type)
            substats.append(
                parse_stat(
                    stat,
                    'substat',
                    item_grade,
                    item_level))

    item = {}
    item['name'] = "GENERATED ITEM"
    item['type'] = item_type
    item['set'] = item_set
    item['level'] = item_level
    item['tier'] = item_tier
    item['enhance'] = 0
    item['is_reforged'] = False
    item['grade'] = item_grade
    item['mainstat'] = mainstat
    item['substats'] = substats

    return item

In [ ]:
def add_new_substat(item, stat_id=None, verbose=False):

    if len(item['substats']) < GRADES[item['grade']]['max_substats']:

        # Get list of current stats, both main and substats
        current_stats = item['substats'] + item['mainstat']

        # Get item type:
        type_ = item['type']
        allowed_subs = convert_int_to_string(TYPES[type_]['substat'])

        if stat_id is not None:

            if check_valid_subs(
                    stat_id):  # Check if the provided stat id is valid

                if str(stat_id) not in allowed_subs:
                    print("This substat cannot be added to this item!")
                    return item

                # Check if provided stat is already in the current pool
                elif any(int(stat_id) == s['id'] for s in current_stats):
                    print("Stat already exists in the item!")
                    return item

                else:
                    new_stat = get_stat_by_id(int(stat_id))

        else:
            new_stat = get_non_overlapping_stat(
                current_stats, 'substat', item['type'])

        # Parse the new stats
        parsed_new_stat = parse_stat(
            new_stat, 'substat', item['grade'], item['level'])
        item['substats'].append(parsed_new_stat)

        # Print Confirmation
        if verbose:
            text = format_stat(parsed_new_stat)
            print(f'New Substat Added: {text}!')
            print('---')

    else:
        print('Item already has max number of substats!')

    return item

In [ ]:
def print_item(item):
    if item:
        # print(item['name'])
        print('---')
        print(
            f"+{item['enhance']} {item['grade']} {item['type']} (iLvL {item['level']})")
        print(f"{item['set']} Set")
        # print(f"iLvl: {item['level']}")

        print('---')
        print('MAIN STAT:')
        print(
            format_stat(
                item['mainstat'][0],
                show_reforged=not item['is_reforged'],
                mainstat=True))

        print('---')
        if len(item['substats']) == 1:
            print('SUBSTAT:')
        else:
            print('SUBSTATS:')
        for s in item['substats']:
            print(format_stat(s, show_reforged=not item['is_reforged']))

        gear_score = calculate_gear_score(item)
        print('---')
        if item['is_reforged']:
            print(f'GEAR SCORE: {gear_score[0]}')
        else:
            print(f'GEAR SCORE: {gear_score[0]} ({gear_score[1]})')
    else:
        return False

In [ ]:
def enhance_mainstat(item):
    main_stat_id = str(item['mainstat'][0]['id'])  # MainStat ID
    enhance = item['enhance']  # Current enhancement level
    multiplier = [
        1.2,
        1.4,
        1.6,
        1.8,
        2,
        2.2,
        2.4,
        2.6,
        2.8,
        3,
        3.3,
        3.6,
        3.9,
        4.25,
        5]
    # Base value of the mainstat for given item grade and tier
    base_val = STATS[main_stat_id]['vars']['mainstat']['values'][item['grade']
                                                                 ][item['tier'] - 5]

    # Assign value based on multiplier and current enhancement level
    item['mainstat'][0]['values'][0][1] = round(base_val * multiplier[enhance])

    return True


def enhance_random_substat(item, verbose=False):
    if 'substats' in item and item['substats']:
        # Choose one of the substats to enhance
        random_substat = random.choice(item['substats'])
        # Get the id of this chosen substat
        random_substat_id = str(random_substat['id'])

        # Get possible roll values along with their rates for this substat
        substat_values = STATS[random_substat_id]['vars']['substat']['values'][item['grade']][item['tier'] - 5]
        substat_rates = STATS[random_substat_id]['vars']['substat']['rates'][item['grade']][item['tier'] - 5]

        try:
            # Get a random value based on the rates
            enhance_value = random.choices(substat_values, substat_rates)[0]
        except IndexError:
            # Handle the case where random.choices doesn't return a value
            return False

        # Add this value to our chosen random substat
        for s in item['substats']:
            if random_substat_id == str(s['id']):
                s['values'][0][1] += enhance_value
                s['rolled'] += 1
                s['reforge_increase'] = get_reforge_increase(
                    random_substat_id, 'substat', s['rolled'])

                # Print Confirmation
                if verbose:
                    text = '+' + \
                        STATS[random_substat_id]['text'] + ' Increase!'
                    text = text.replace(
                        '<A>', str(enhance_value)).replace(
                        '<B>', '')
                    print(text)

                return True
    return False


def enhance_item(item, verbose=False):

    # Current enhance level
    enhance = item['enhance']

    if verbose:
        if enhance == 14:
            print('Max enhance level reached!')

    if enhance < 15:  # This is the max enhance level

        # Upgrade Main Stat
        enhance_mainstat(item)

        # Substat Add Conditions
        # Normal Gear:
        if item['grade'] == 'Normal' and (
                enhance == 2 or enhance == 5 or enhance == 8 or enhance == 11):
            add_new_substat(item, verbose=verbose)

        # Good Gear:
        elif item['grade'] == 'Good' and (enhance == 5 or enhance == 8 or enhance == 11):
            add_new_substat(item, verbose=verbose)

        # Rare Gear:
        elif item['grade'] == 'Rare' and (enhance == 8 or enhance == 11):
            add_new_substat(item, verbose=verbose)

        # Heroic Gear:
        elif item['grade'] == 'Heroic' and (enhance == 11):
            add_new_substat(item, verbose=verbose)

        # For remaining cases
        elif ((enhance + 1) % 3) == 0:  # Enhance only at 3 level increments
            enhance_random_substat(item, verbose=verbose)

        # Increase enhance level by 1
        item['enhance'] += 1

        return item

    else:
        print('Item already at max enhance level!')
        return item

In [ ]:
def calculate_gear_score(item):
    gear_score = 0  # initialize unreforged gear score
    reforged_score = 0  # initialize reforged gear score

    # Calculate gear score for each substat
    for s in item['substats']:
        s_id = str(s['id'])
        s_val = s['values'][0][1]
        gear_score += s_val * STATS[s_id]['gscore']

        s_val_ref = s_val + s['reforge_increase']
        reforged_score += s_val_ref * STATS[s_id]['gscore']

    return [round(gear_score), round(reforged_score)]

In [ ]:
def reforge_stat(parsed_stat, stat_type='substat'):
    if stat_type == 'substat':
        reforged_value = parsed_stat['values'][0][1] + \
            parsed_stat['reforge_increase']
    else:
        reforged_value = parsed_stat['reforge_increase']
    return reforged_value

In [ ]:
def reforge_item(item):

    if item['is_reforged']:
        print("Cannot reforge item that has already been reforged.")
        return item

    if item['level'] != 85:
        print("Cannot reforge item that is not iLvl 85.")
        return item

    if item['enhance'] < 15:
        print("Cannot reforge item if it has not been enhanced to +15.")
        return item

    else:

        # Set main stat to reforged stat
        item['mainstat'][0]['values'][0][1] = item['mainstat'][0]['reforge_increase']

        # Set each substats to reforged value
        for s in item['substats']:  # iterate through each substat
            s['values'][0][1] = reforge_stat(s)

        # Set reforged status to True
        item['is_reforged'] = True

        # Change item_level to 90
        item['level'] = 90

        # Print confirmation:
        print("Item has been reforged!")

    return item

In [ ]:
# This item modifies a particular stat in the item, stat number is the
# index number of the stat to be modified

def modify_item(item, stat_index=None, mod_stat_id=None,
                mod_type='greater', verbose=False):

    # Check if item is fully enhanced:
    if item['enhance'] != 15:
        print("Cannot modify item unless it has been fully enhanced to +15.")
        return item

    if (stat_index is not None) and (isinstance(
            stat_index, (str, int))) and (0 < stat_index < 5):
        # Get correct indexing for our code (start from 0)
        stat_index = int(stat_index) - 1
    else:
        print("Please provide a valid integer value between 1 and 4 for the stat index.")
        return item

    # Check if any of the other substats have been modified (they will have 'modded' = True)
    # If so, we cannot proceed with modification, as modification is only
    # allowed on the substat where we've applied one mod
    if any(substat.get('modded', False)
           for i, substat in enumerate(item['substats']) if i != stat_index):
        print("Cannot modify substat when another substat has been modded.")
        return item

    # Check if correct stat_index has been provided:
    if (mod_stat_id is not None) and (isinstance(mod_stat_id, (int, str))) and (
            0 <= int(mod_stat_id) <= len(STATS) - 1):
        # Get correct indexing for our code (start from 0)
        mod_stat_id = str(mod_stat_id)
    else:
        print("Please provide an integer value between 0 and 10 for mod_stat_id.")
        return item

    # Check for correct mod_type (it can either be 'greater' or 'lesser')
    if mod_type is not None and mod_type not in ['greater', 'lesser']:
        print("Invalid mod_type provided. Defaulting to Greater Gem.")
        mod_type = 'greater'

    # Get item type and get allowed subs
    type_ = item['type']
    allowed_subs = convert_int_to_string(TYPES[type_]['substat'])

    # Check if the mod stat is applicable in the pool
    if mod_stat_id not in allowed_subs:
        print("This substat cannot be added to this item type")
        return item
    else:
        # Check if the mod stat already exists on the item.
        current_stats = item['substats'] + item['mainstat']

        if any(str(stat['id']) == mod_stat_id for i,
               stat in enumerate(current_stats) if i != stat_index):
            print("Cannot add a substat that already exists on the item!")
            return item
        else:
            # Get the count of how many times this substat has rolled or use 0
            # if it doesn't exist
            roll_count = item['substats'][stat_index].get('rolled', 0)
            # Get a new stat based on provided id
            new_stat = get_stat_by_id(mod_stat_id)
            modified_stat = parse_stat(
                new_stat,
                'substat',
                item['grade'],
                item['level'],
                mod=True,
                rolled=roll_count,
                mod_type=mod_type)  # Parse the stat based on modded values

            # Replace the substat with the new modded stat
            item['substats'][stat_index] = modified_stat
            # Print confirmation
            print("Item has been modded with a new substat!")

    return item

# TESTING

In [ ]:
item1 = create_random_item(
    mainstat_id='1',
    item_set='Speed',
    item_type='Ring',
    item_grade='Epic')
print_item(item1)

In [ ]:
print_item(enhance_item(item1, verbose=True))

In [ ]:
print_item(modify_item(item1, 1, 6))

In [ ]:
print_item(reforge_item(item1))

In [ ]:
# For testing purposes
# import sys

# # Save the original stdout to restore it later
# original_stdout = sys.stdout

# # Open a file for writing the print log
# with open('print_log.txt', 'w') as f:
#     # Redirect stdout to the file
#     sys.stdout = f

#     iters = 10000
#     enhance_tries = 16
#     items = []

#     for i in range(1,iters):
#         item = create_random_item(item_grade = 'Epic')
#         for j in range(1,enhance_tries):
#             enhance_item(item)
#         gear_score = calculate_gear_score(item)
#         if gear_score > 100:
#             items.append(item)
#         # Restore the original stdout
#     sys.stdout = original_stdout

In [ ]:
def has_mainstat(item, ids=[]):

    mainstat = []
    mainstat.extend(item['mainstat'])

    list_ids = [m['id'] for m in mainstat]

    for id in ids:
        if id not in list_ids:
            return False
    return True

In [ ]:
def has_substats(item, ids=[]):

    substats = []
    substats.extend(item['substats'])

    list_ids = [s['id'] for s in substats]

    for id in ids:
        if id not in list_ids:
            return False
    return True

In [ ]:
# def to_view(item):

# 	view_dict = {}
# 	view_dict['name'] = item['name']
# 	view_dict['tier'] = item['tier']
# 	view_dict['type'] = item['type']
# 	view_dict['stats'] = []
# 	view_dict['affixes'] = []
# 	view_dict['legendary'] = []
# 	view_dict['sockets'] = []

# 	for stat in item['stats']:
# 		view_dict['stats'].append(format_affix(stat))
# 	for affix in item['affixes']:
# 		view_dict['affixes'].append(format_affix(affix))
# 	for affix in item['legendary']:
# 		view_dict['legendary'].append(format_affix(affix))
# 	for socket in item['sockets']:
# 		view_dict['sockets'].append(format_affix(socket))

# 	view_dict['main_stat'] = view_dict['stats'].pop(0) #temp
# 	view_dict['sell_value'] = 999
# 	view_dict['durability'] = 100

# 	dmg_icon = 'default'
# 	if item['type'] in CONFIG['offensive']:
# 		dmg_icon = 'offensive'
# 	elif item['type'] in CONFIG['defensive']:
# 		dmg_icon = 'defensive'
# 	view_dict['dmg_icon'] = dmg_icon

# 	return view_dict

In [ ]:
# TO-DO
# Clean Code
# Separate into classes and scripts
# Create json files for data

# FUTURE
# Create Images of items
# Inventory System to save items